In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [4]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.
Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""



In [5]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [6]:
scenarios = ["Successful Registration", "Username Below Minimum Length", "Username Above Maximum Length",
             "UsernameAlreadyRegistered", "Password Below Minimum Length", "Password Above Maximum Length",
             "Password Missing Normal Character", "Password Missing Capitalized Character", 
             "Password Missing Number", "Password Missing Special Character", 
             "Password Same as Username", "Learner chooses to register by email instead of username",
             "EmailWrongFormat", "EmailInexistent"]

In [7]:
prompt_uc = """
Registry
Main flow
Step 1: Learner fills in the username field by a valid username, a valid username must be over 8 characters and below 30 characters.
Step 2: Learner fills in the password field by a valid password, a valid password must be over 8 characters and below 30 characters, contains at least one of each kind: a normal character, a capitalize character, a number and a special character. A valid password cannot be the same as the username.
Step 3: Learner press "Registry" button
Step 4: System redirects learner to Home page

Alternative flow 1: Learner register by email
At step 1 of the basic flow: Learner clicks on the Mail icon
Step 2: Learner fills in the email field by a valid email, a valid email must be an exist email with the right format
Go back to step 2 in the basic flow and continue with the steps from step 2.

Exception flow 1: Learner enters invalid username
At step 1 of the basic flow: Learner fills in the username field by an invalid username
Go back to step 2 in the basic flow and continue with the steps from step 2.
At step 4 of the basic flow: System shows a notification that the username is invalid

Exception flow 2: Learner enters invalid password
At step 2 of the basic flow: Learner fills in the password field by an invalid password
Go back to step 3 in the basic flow and take only step 3.
At step 4 of the basic flow: System shows a notification that the passowrd is invalid

Exception flow 3: Learner enters an existed username
At step 1 of the basic flow: Learner fills in the username field by an username that has been registered
Go back to step 2 in the basic flow and continue with the steps from step 2.
At step 4 of the basic flow: System shows a notification that this username has been registered and ask the learner to choose another username.

Exception flow 4: Learner enters invalid email
At step 2 of the alternative flow 1: Learner fills in the email field by an invalid email, an invalid email must be an inexist email or email with the wrong format
Go back to step 3 in the alternative flow 1 and continue with the steps from step 3.
At step 4 of the basic flow: System shows a notification that the mail is invalid
"""

In [8]:
for scenario in scenarios:
        promptTestCase = [
                { "role": "system", "content": SYSTEM_PROMPT_8},
                { "role": "user", "content": scenario + "\n" + prompt_uc}
                ]
        gpt_response = ask(promptTestCase, client, model)
        json_data = json.loads(gpt_response)
        # write_json_to_txt_raw(json_data, scenario + "-Raw" + ".txt")
        print(gpt_response)
        promptSelfRef = [
        { "role": "system", "content": SELF_REF},
        { "role": "user", "content": scenarios[2] + "\n" + gpt_response}
        ]
        final_response = ask(promptSelfRef, client, model)
        print(final_response)
# json_fin = json.loads(final_response)
# write_json_to_txt(json_fin, scenarios[2] + ".txt")

{
  "Test Case 1": {
    "testCaseName": "Successful Registration with Username and Password",
    "objective": "Verify that a learner can successfully register using a valid username and password and is redirected to the Home page.",
    "testSteps": [
      "Step 1: Navigate to the Registry page.",
      "Step 2: Fill in the username field with a valid username (over 8 characters and below 30 characters).",
      "Step 3: Fill in the password field with a valid password (over 8 characters and below 30 characters, contains at least one normal character, one capitalize character, one number, and one special character, and is not the same as the username).",
      "Step 4: Press the 'Registry' button."
    ],
    "expectedResult": "The learner is redirected to the Home page after successful registration.",
    "explanation": "This test case ensures that the registration process works correctly when valid credentials are provided."
  },
  "Test Case 2": {
    "testCaseName": "Successful 